All shapefiles collected from https://vgin.vdem.virginia.gov/
- 3793 Road features: Virginia Road Centerlines (RCL) 70+ mph
- 9113 Railway features: Virginia Railroad

In [54]:
from shapely import intersection
from shapely.ops import transform
from shapely.geometry import MultiPoint
import pyproj, os, shapefile, shapely
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

root_src = r'C:\Users\Doug PC\Documents\DoD reference\GLAP\GLGD4003 Spatial Data Structures\Module2\Homework'
railroads = os.path.join(root_src,r'VGIN_RAIL\Railroad.shp') # 9113 features
roads = os.path.join(root_src,r'VirginiaRoadCenterline\VirginiaRoad_70plus.shp') # 3793 features

wgs84 = pyproj.CRS('EPSG:4326')
Lambert_Conformal_Conic_Virginia = pyproj.CRS('EPSG:3968')
project = pyproj.Transformer.from_crs(Lambert_Conformal_Conic_Virginia, wgs84, always_xy=True).transform

In [67]:
def shapefileToShapely(filename):
    shp = shapefile.Reader(filename)
    geoms = [shapely.geometry.LineString(rec.shape.points) for rec in shp.shapeRecords()]
    geoms_shapely = np.array([transform(project, L) for L in geoms])
    return geoms_shapely

def queryIndex(geoms,strtree):
    indices = strtree.query(geoms,predicate='intersects')
    result_pairs = np.array([geoms.take(indices[0]),strtree.geometries.take(indices[1])]).T.tolist()
    intersect_pts = [intersection(a,b) for a,b in result_pairs]
    return intersect_pts

def findIntersectPoints(line1,line2):
    pnts = list()
    for i in line1:
        for j in line2:
            if i.intersects(j):
                pnts.append(intersection(i,j))
    return pnts

In [59]:
print('Import Railroad Shapefile')
%timeit rail_geoms_shapely = shapefileToShapely(railroads)
rail_geoms_shapely = shapefileToShapely(railroads)
print('Import Road Shapefile')
%timeit road_geoms_shapely = shapefileToShapely(roads)
road_geoms_shapely = shapefileToShapely(roads)

1.89 s ± 25.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
937 ms ± 20.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [60]:
print('Build Index')
%timeit roads_strtree = shapely.STRtree(road_geoms_shapely)
roads_strtree = shapely.STRtree(road_geoms_shapely)

Build Index
494 µs ± 15.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [66]:
print('Query index')
%timeit intersect_pts = queryIndex(rail_geoms_shapely,roads_strtree)
intersect_pts = queryIndex(rail_geoms_shapely,roads_strtree)

Query index
14.6 ms ± 141 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [42]:
%timeit intersect_pairs = findIntersectPoints(road_geoms_shapely,rail_geoms_shapely)

KeyboardInterrupt: 

In [50]:
cmap_red = ListedColormap(['red'], name='allred')
cmap_green = ListedColormap(['green'], name='allred')

gdf1 = gpd.GeoDataFrame(geometry=road_geoms_shapely,crs='epsg:4326')
gdf2 = gpd.GeoDataFrame(geometry=rail_geoms_shapely,crs='epsg:4326')
gdf3 = gpd.GeoDataFrame(geometry=gpd.GeoSeries(MultiPoint(np.array(intersect_pts))),crs='epsg:4326')
ax = gdf1.plot(cmap=cmap_red)
gdf2.plot(ax=ax, cmap=cmap_green)
gdf3.plot(ax=ax)